In [1]:
import yaml
import torch
import random

from torch.utils.data import Subset

from model.model import Model
from model.trainer import Trainer
from model.test import Test
from model.model_analyze import ModelAnalyze

from data_pre.dataset import TrainValidDataset, TestDataset, T5TrainValidDataset, T5TestDataset
import pandas as pd


random_seed = 42
random.seed(random_seed)

pd.set_option('display.max_colwidth', None)

# Model & Tokenizer & Dataset 생성

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with open('/root/upstage-nlp-summarization-nlp2/config.yaml', "r") as file:
    config = yaml.safe_load(file)

# Model & Tokenizer
model_instance = Model(config, device)

tokenizer = model_instance.getT5Tokenizer()
model = model_instance.getT5Model(tokenizer)

# dataset
train_dataset = T5TrainValidDataset(config, tokenizer, is_train=True)
val_dataset = T5TrainValidDataset(config, tokenizer, is_train=False)
test_dataset = T5TestDataset(config, tokenizer)

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Trainer 학습

In [3]:
trainer = Trainer(config, model, train_dataset, val_dataset, tokenizer).get_trainer()
trainer.train()

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/myvenv/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tjsgh2770 (prefer_leee). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,0.384000,0.599953,0.209205,0.046261,0.205579


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=779, training_loss=2.2995037646899634, metrics={'train_runtime': 544.337, 'train_samples_per_second': 22.885, 'train_steps_per_second': 1.431, 'total_flos': 9066267722907648.0, 'train_loss': 2.2995037646899634, 'epoch': 1.0})

# Test 추론

In [4]:
best_model_path = trainer.state.best_model_checkpoint

test_inst = Test(config, test_dataset, tokenizer, device)
model = test_inst.getT5Model(best_model_path)

model_name = config['model']['t5']
result_df = test_inst.testModel(model, model_name)
result_df

100%|██████████| 16/16 [01:01<00:00,  3.83s/it]


,fname,summary
0,test_0,모든 직원에게 내부 메모로 전달되어야 합니다. 그들은 즉시 메시지를 사용하고 있습니다.
1,test_1,대중교통을 이용하는 것이 환경에 더 좋을 수 있다. 그들은 차를 가지고 있는 자유가 그리워질 거라고 생각한다.
2,test_2,케이트는 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신청했다고 합니다. 그들은 아이들의 양육권에 대해 이야기하고 있습니다.
3,test_3,브라이언은 브라이언의 생일을 축하하고 함께 술 한 잔 하자고 제안한다.
4,test_4,올림픽 스타디움은 6월에 완공될 예정이다. 그들은 외국인 방문객을 위해 영어로 번역된 표지판과 점프 피트를 많이 설치했다.
...,...,...
494,test_495,잭은 헤이와 찰리에게 비디오 게임을 보여주고 싶어합니다.
495,test_496,아내와 아내가 처음으로 레코드 플레이어를 구입했을 때부터 컨트리 음악에 관심을 가지게 되었습니다. 그들은 CBC에서 제안받은 프로그램으로인해 자신의 도서관 설립을하게 됩니다.
496,test_497,앨리스는 앨리스에게 세탁기와 건조기의 사용법을 알려준다. 하지만 그들은 모두 엄마가 아이들을 위해 빨래를 해주지 않는다고 말한다.
497,test_498,스티브는 스티브에게 최근에 살 곳을 찾고 있다고 말한다. 그들은 이미 집을 찾았으나 여전히 분류된 광고를 찾는 중이다.


# Model 성능 분석

In [6]:
model_path = '/root/upstage-nlp-summarization-nlp2/results/checkpoint-2337'
model_analyze = ModelAnalyze(config, tokenizer, device)
result_df = model_analyze.get_result(val_dataset, model_path)
result_df

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
100%|██████████| 16/16 [00:40<00:00,  2.52s/it]


,input_text,generated_text,label,rouge_1,rouge_2,rouge_l,rouge_mean
187,"#Person1# : 음... 내 식기는 어디에 있나요? \n#Person2# : 손으로 먹습니다. 빵을 집어서 숟가락처럼 음식을 떠먹으세요. \n#Person3# : 이 요리, 질보는 양고기와 감자가 들어있습니다. \n#Person1# : 허허! 정말 맛있네요! 꿀 와인 좀 주실 수 있나요? 그럼, 클라이드, 너는 무엇을 공부하니? \n#Person2# : 국제법과 비즈니스입니다. 우리 가족은 자체 회사를 운영하고 있어요. \n#Person3# : 클라이드는 첫 번째 흑인 대통령이 되고 싶어합니다. \n#Person2# : 첫 번째 흑인 대통령이 될 겁니다. 나는 론알드 레이건을 롤 모델로 삼고 있어요.","<usr>#Person1# 은 #Person2# 에게 클라이드의 식기, 질보는 양고기와 감자, 그리고 꿀 와인을 소개합니다.</s>","#Person1# 과 클라이드, #Person3# 이 함께 식사를 하는데 클라이드가 첫 번째 흑인 대통령이 되고 싶다고 말합니다.</s>",0.0645,0.0000,0.0645,0.0430
323,"#Person1# : 실례합니다. 저는 새로 이사 온 이웃, 잭입니다.\n#Person2# : 오, 네?\n#Person1# : 식료품점을 찾고 있는데, 여기 주변에 있나요?\n#Person2# : 네, 파인 스트리트에 몇 군데 있습니다.\n#Person1# : 알겠습니다. 그럼 여기 근처에 세탁소는 있나요?\n#Person2# : 음, 쇼핑 센터 맞은편에 있는 것 같습니다.\n#Person1# : 감사합니다.\n#Person2# : 그런데, 쇼핑 센터 안에 이발소도 있습니다.\n#Person1# : 이발소라고요?!",<usr>#Person1# 은 새로 이사 온 잭에게 식료품점을 찾고 있는지 묻습니다. 잭은 쇼핑 센터 맞은편에 세탁소가 있다고 말합니다.</s>,"#Person2# 는 #Person1# 에게 식료품점, 세탁소, 그리고 이발소의 위치를 알려줍니다.</s>",0.0690,0.0000,0.0690,0.0460
349,"#Person1# : 리사, 안녕!\n#Person2# : 안녕, 넷!\n#Person1# : 보통 출근할 때 차를 타지 않나요?\n#Person2# : 보통은 그렇지만, 날씨가 안 좋을 때는 아니에요.\n#Person1# : 왜죠?\n#Person2# : 비나 눈이 오면 교통이 훨씬 더 복잡해져요.\n#Person1# : 저는 항상 기차를 타요. 운전하는 것보다 훨씬 빠르거든요.\n#Person2# : 저에게도 더 빠르긴 해요. 하지만 그 시간이 항상 편리하지는 않아요.\n#Person1# : 그래도, 기차에서 할 수 있는 일들을 생각해봐요.\n#Person2# : 책이나 신문을 읽는 것 말이죠.\n#Person1# : 아니면 이메일을 답장하는 것. 고맙지만, 제 근무 시간은 이미 충분히 길어요.","<usr> 리사는 #Person1# 에게 출근할 때 차를 타지 않으며, 기차에서 할 수 있는 일들을 생각해보라고 제안한다.</s>",리사와 넷은 <unk>안 좋은 날씨에 어떻게 출근하는지에 대해 이야기한다.</s>,0.0741,0.0000,0.0741,0.0494
259,"#Person1# : 하지만 어떻게 제가 웹트래커에서 일하고 있다는 것을 알았나요?\n#Person2# : LA에 있을 때, 저는 웹트래커의 영업 대표를 만났습니다.\n#Person1# : 그는 당신에게 말하지 않았을 것입니다. 웹트래커에서는 저를 아는 사람이 몇 명 없습니다.\n#Person2# : 영업 대표는 당신의 것과 똑같은 녹색 노트패드를 가지고 있었습니다. 그는 웹트래커의 모든 사람들이 그것을 사용한다고 말했습니다.\n#Person1# : 믿을 수 없어요. 당신이 친 함정에 빠졌다는 것을 믿을 수 없어요.\n#Person2# : 우리 둘 다 뱀일지라도, 데이브, 제가 좀 더 낫습니다. 뱀들 중에서, 저는 가장 교활합니다.",<usr> 데이브와 #Person1# 은 웹트래커의 영업 대표를 만났다. 그들은 서로의 장점을 비교한다.</s>,Dave는 #Person2# 가 그가 웹트래커에서 일하고 있다는 것을 추론할 수 있다는 사실에 놀란다.</s>,0.0769,0.0000,0.0769,0.0513
77,"#Person1# : 안녕, 케빈!\n#Person2# : 안녕, 로라, 오랜만이네! 요즘 뭐하고 지냈어?\n#Person1# : 별로. 확신할 수 있어. 너는?\n#Person2# : 나도 비슷한데, 큰 소식이 있어.\n#Person1# : 어서 말해. 너무 긴장돼서 죽을 것 같아.\n#Person2# : 아니, 정말로, 너는 이 몇 주 동안 뭐하고 있었어? 마지막에 널 봤을 때, 너는 새로운 일자리를 찾고 있었잖아.\n#Person1# : 음, 사실 그건 아냐. 나는 직장을 옮기려고 생각하고 있었어. 다행히도, 회계 부서에서 새로운 직책을 제안해줬어.\n#Person2# : 회사 생활에서 승진했네.\n#Person1# : 과장이 아니라, 나 정말 기뻐. 나는 한동안 승진을 희망하고 있었어. 그래서 드디어 승진이 결정되었을 때, 나는 안도했어. 사실, 그게 내가 새로운 일자리를 찾고 있던 이유야. 그들이 내 노력을 인정하지 않는다면, 나는 더 이상 그곳에서 일하고 싶지 않았어.\n#Person2# : 그렇지, 가끔 너의 최선을 다해도 다른 사람들은 너가 존재하는지 모르는 것 같아. 월급이 좀 올라간 거면 좋겠어.\n#Person1# : 합리적인 인상을 받았어, 이제 나에 대해 충분히 얘기했으니. 너의 소식을 듣고 싶어 죽겠어.\n#Person2# : 나 결혼하려고 해.\n#Person1# : 아니, 너는 결혼하지 않겠다고 했잖아.\n#Person2# : 그건 그때의 이야기고, 이건 지금의 이야기야. 너는 반드시 안드레아를 만나봐야 해, 그녀는 정말 대단해!\n#Person1# : 이건 모두 나에게 새로운 소식이야. 너가 데이트를 하고 있다는 것조차 몰랐어.\n#Person2# : 우리는 그렇지 않았어. 우리는 지금 두 주 동안 만나고 있었어.\n#Person1# : 그리고 너는 결혼하려고 해?\n#Person2# : 알아. 어쩔 수 없어. 그녀가 바로 내가 찾던 그 사람이라는 것을 알고 있어.\n#Person1# : 그럼, 축하해! 정말 대단하다!\n#Person2# : 고마워, 너도 그렇게 생각한다니 기쁘다.",<usr> 케빈은 로라에게 그녀가 새로운 일자리를 찾고 있다고 말한다. 케빈은 그녀가 직장을 옮기려고 생각하고 있었지만 다행히도 회계 부서에서 새로운 직책을 제안받았기 때문에 승진을 하게 되었다. 그녀는 또한 안드레아를 만나기 위해 그녀를 만나고 싶어한다.</s>,"로라와 케빈은 오랜만에 만나 서로의 최근 소식을 공유한다. 로라는 직장 승진과 월급 인상을 받았고, 케빈은 안드레아와 2주 동안 데이트한 후 결혼했다.</s>",0.0769,0.0000,0.0769,0.0513
...,...,...,...,...,...,...,...
472,"#Person1# : 메리, 우리가 처음 만났던 날을 기억하니?<unk>\n#Person2# : 절대 잊지 못해! 나 혼자서 카페테리아에 앉아 있었어...<unk>\n#Person1# : 그리고 나는 네 옆에 앉아도 되는 지 물어봤었지...<unk>\n#Person2# : 아, 마치 어제 일 같아.",<usr> 메리는 #Person1# 에게 카페테리아에서 처음 만났던 날을 회상한다.</s>,메리와 #Person1# 은 처음 만났던 날을 회상한다.</s>,0.6667,0.5000,0.6667,0.6111
452,#Person1# : 실례합니다. 가장 가까운 영화관에 어떻게 가는지 알려주실 수 있나요?<unk>\n#Person2# : 두 번째 신호등에서 왼쪽으로 가세요. 찾으실 수 있을 거예요.<unk>\n#Person1# : 거기에 가는 데 오래 걸리나요?<unk>\n#Person2# : 아니요. 전혀 멀지 않아요.<unk>\n#Person1# : 정말 감사합니다.<unk>\n#Person2# : 괜찮아요. 좋은 하루 보내세

In [7]:
result_df.to_csv('/root/upstage-nlp-summarization-nlp2/submits/temp.csv', index=False)